# Sample Complexity Gap

This notebook aims to demonstrate the stated sample complexity gap in **Why Are Convolutional Networks More Sample Efficient Than Fully-Connected Nets? by Zhiyuan Li, Yi Zhang and Sanjeev Arora** [1]. We set up an experiment in which we should see the gap as an increasing polynomial curve of degree less than two.

## 1. Methods

For a given input dimension $d$, we seek the number $|S_{tr}|$ of training samples needed for a model to reach $\epsilon=0.9$ test accuracy. Then we plot the difference of training samples needed between a Convolutional Neural Network and a Fully Connected Neural Network for increasing values of $d$.

### Data

The inputs are $3\times k \times k$ RGB images for $k\in \mathbb{N}$, yielding input dimensions $d\in \{..., 192, 243, 300, 363, ...\}$. We create full training set of 10000 images and a test set of 10'000 and we ask "the first *how-many* training samples are needed to reach $90\%$ test accuracy if we train until convergence"? The training sets are constructed in the following manner.
+ Entry-wise independent Gaussian (mean 0, standard deviation 1)

We explore two different labelling functions 
\begin{equation}
h_1=\mathbb{1}[\sum_{i\in R} x_i > \sum_{i \in G}x_i] \quad\mathrm{ and }\quad h_2=\mathbb{1}[\sum_{i\in R} x_i^2 > \sum_{i \in G}x_i^2].
\end{equation}

### Models

1. 2-layer CNN
    + Convolution: One kernel per input channel of size 3x3, 10 output channels, stride size 1, and padding of 1, and bias
    + Activation function
    + Max pooling, kernel size 2x2, stride 2
    + Fully connected layer (160 in, 1 out) with bias
    + Sigmoid  
2. 2-layer "Dumb" CNN. (DCNN)
    + Convolution: One kernel per input channel of size 3x3, 2 output channels, stride size 1, and padding of 0, and bias
    + Activation function
    + Global average pooling, kernel size 8x8
    + Fully connected layer (160 in, 1 out) with bias
    + Sigmoid
2. 2-layer FCNN 
    + Fully connected layer (192 in, 3072 out) with bias
    + Activation function 
    + Fully connected layer (3072 in, 1 out) with bias
    + Sigmoid
    
We try both ReLU and Quadratic activation functions. 

### Training algorithm
+ Stochastic Gradient Descent with batch size 64
+ BCELoss
+ Learning rate $\gamma = 0.01$
+ Stopping criterion: At least 10 epochs AND Training loss < 0.01 AND Rolling avg. of rel. change in training loss < 0.01 (window size 10). OR 500 epochs.

### Model Evaluation
+ The model $M$ prediction is $\mathbb{1}[M(x)>0.5]$. Test accuracy is the percentage of correct predictions over the test set.

### Search algorithm

We seek the number of training samples needed to reach a fixed test accuracy using a kind of bisection algorithm.
1. Initial training run on 5000 samples.
    + If test accuracy > 0.9, take half step towards 0 -> 2'500
    + If test accuracy <= 0.9 take half step towards 10'000 -> 7500
2. Reload initial weights and retrain. Make quarter step.

This is repeated $10$ times with different weight initialisations in case the test-accuracy curves are not monotonically increasing due to noise.

## 2. Experiment

Optional Arguments
- -min/max (int) The min/max image side lengths. Default 4/14.
- -f (str) The output .json file name (must contain an empty {}). Default week2_out.json
- -e (int) The maximum number of epochs to run the programme for. Default 100.
- -p (1/2) The p-norm used in the labelling function. Default 2.

In [6]:
# Uncomment to run the experiment
!python week2.py -min 4 -max 14 -f week2_out.json -e 20 -p 2

^C


## 3. Results

In [ ]:
import matplotlib.pyplot as plt

# Load data
file_path = 'week2_out.json'
with open(file_path, 'r') as json_file:
    test_acc = load(json_file)

# Create sorted dictionary out of saved data
new_dict = {}
for key in test_acc.keys():
    new_dict[int(key)]=test_acc[key]
sorted_dict = dict(sorted(new_dict.items()))

# Convert the dictionary to a format suitable for plotting
x_values = np.sort([int(key) for key in sorted_dict.keys()])  # Convert keys to integers

# Model names
names = ["CNN+ReLU", "CNN+Quad", "DCNN+ReLU", "DCNN+Quad", "FCNN+ReLU", "FCNN+Quad"]

# For every model, make line plot
for i, name in enumerate(names):
    
    # The accuracy values for increasing number of samples
    y_values = [value[i] for value in sorted_dict.values()]

    # Create a line plot
    plt.plot(x_values, y_values, marker='.', linestyle='-', label=name)
    
# Plot graphics
plt.xlabel('Input dimension')
plt.ylabel('Training set size')
plt.title(f'Training set size required to reach {epsilon} test accuracy')
plt.legend()
plt.grid(True)

# Show the plot
plt.show()

1. [Why Are Convolutional Nets More Sample-Efficient than Fully-Connected Nets?](https://arxiv.org/abs/2010.08515) Zhiyuan Li, Yi Zhang, Sanjeev Arora, 2021